# 2.3) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Import statements
Following packages are necessary:

In [1]:
import random
import utils

import tensorflow as tf
import numpy as np
import segmentation_models as sm

from unet import UNet
from unet import preprocess_input as pre_une
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.callbacks import CSVLogger, ModelCheckpoint
from sklearn.model_selection import train_test_split
from segmentation_models.losses import dice_loss
from segmentation_models.metrics import iou_score
from lastengen import LASTENSequence
from matplotlib import pyplot as plt

Segmentation Models: using `tf.keras` framework.


# Checks
The version of tensorflow as well as the GPU support are checked.

In [2]:
print(tf.__version__)
#physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], True)
tf.config.experimental.list_physical_devices('GPU')

2.0.0


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

# Data
Import training images

In [3]:
path_train = r'D:\LASTEN\train'
path_validation = r'D:\LASTEN\validation'
path_test = r'D:\LASTEN\test'

#path_train = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/train'
#path_validation = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/validation'
#path_test = r'/home/julian/Documents/Studium/MT-Masterarbeit/Data/LASTEN/test'

width = 768 #384
height = 768 #384

# Seeding
Seeds are set to ensure reproducible training.

In [4]:
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)
random.seed(SEED)

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [5]:
model = UNet(filters=32, input_shape=(height, width, 1))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 768, 768, 1) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 768, 768, 32) 288         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 768, 768, 32) 128         conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 768, 768, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [6]:
image_ids = None

train_gen = LASTENSequence(path_train,
                           batch_size=2,
                           image_ids=image_ids,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=2,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [7]:
save_path = "experiments/2_3_segmentation"

logger = CSVLogger(save_path + "/result.log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5",
                          period=10)

callbacks = [logger, checker]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [8]:
model.fit_generator(train_gen,
                    epochs=300,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/300
84/84 [==============================] - 68s 807ms/step - loss: 0.7804 - accuracy: 0.9298 - iou_score: 0.1258 - val_loss: 0.9683 - val_accuracy: 0.9817 - val_iou_score: 0.0161
Epoch 2/300
84/84 [==============================] - 52s 615ms/step - loss: 0.4960 - accuracy: 0.9879 - iou_score: 0.3404 - val_loss: 0.7666 - val_accuracy: 0.9848 - val_iou_score: 0.1334
Epoch 3/300
84/84 [==============================] - 52s 615ms/step - loss: 0.3237 - accuracy: 0.9900 - iou_score: 0.5118 - val_loss: 0.8093 - val_accuracy: 0.9830 - val_iou_score: 0.1074
Epoch 4/300
84/84 [==============================] - 51s 609ms/step - loss: 0.2677 - accuracy: 0.9907 - iou_score: 0.5782 - val_loss: 0.5703 - val_accuracy: 0.9860 - val_iou_score: 0.2880
Epoch 5/300
84/84 [==============================] - 52s 616ms/step - loss: 0.2499 - accuracy: 0.9909 - iou_score: 0.6010 - val_loss: 0.2823 - val_accuracy: 0.9910 - val_iou_score: 0.5614
Epoch 6/300
84/84 [==============================] - 52s 616

84/84 [==============================] - 51s 603ms/step - loss: 0.1978 - accuracy: 0.9922 - iou_score: 0.6705 - val_loss: 0.1117 - val_accuracy: 0.9956 - val_iou_score: 0.7998
Epoch 88/300
84/84 [==============================] - 51s 612ms/step - loss: 0.1959 - accuracy: 0.9923 - iou_score: 0.6736 - val_loss: 0.1050 - val_accuracy: 0.9959 - val_iou_score: 0.8104
Epoch 89/300
84/84 [==============================] - 51s 610ms/step - loss: 0.1972 - accuracy: 0.9923 - iou_score: 0.6717 - val_loss: 0.1197 - val_accuracy: 0.9952 - val_iou_score: 0.7867
Epoch 90/300
84/84 [==============================] - 51s 613ms/step - loss: 0.1925 - accuracy: 0.9924 - iou_score: 0.6782 - val_loss: 0.1062 - val_accuracy: 0.9959 - val_iou_score: 0.8086
Epoch 91/300
84/84 [==============================] - 51s 613ms/step - loss: 0.1933 - accuracy: 0.9924 - iou_score: 0.6771 - val_loss: 0.1019 - val_accuracy: 0.9960 - val_iou_score: 0.8156
Epoch 92/300
84/84 [==============================] - 51s 606ms/step

84/84 [==============================] - 51s 605ms/step - loss: 0.1883 - accuracy: 0.9926 - iou_score: 0.6844 - val_loss: 0.0913 - val_accuracy: 0.9964 - val_iou_score: 0.8333
Epoch 174/300
84/84 [==============================] - 51s 612ms/step - loss: 0.1870 - accuracy: 0.9927 - iou_score: 0.6860 - val_loss: 0.1013 - val_accuracy: 0.9960 - val_iou_score: 0.8168
Epoch 175/300
84/84 [==============================] - 51s 609ms/step - loss: 0.1906 - accuracy: 0.9925 - iou_score: 0.6810 - val_loss: 0.0926 - val_accuracy: 0.9963 - val_iou_score: 0.8311
Epoch 176/300
84/84 [==============================] - 51s 610ms/step - loss: 0.1886 - accuracy: 0.9926 - iou_score: 0.6838 - val_loss: 0.1071 - val_accuracy: 0.9958 - val_iou_score: 0.8072
Epoch 177/300
84/84 [==============================] - 51s 613ms/step - loss: 0.1910 - accuracy: 0.9925 - iou_score: 0.6804 - val_loss: 0.0937 - val_accuracy: 0.9963 - val_iou_score: 0.8292
Epoch 178/300
84/84 [==============================] - 51s 608ms

84/84 [==============================] - 51s 607ms/step - loss: 0.1846 - accuracy: 0.9928 - iou_score: 0.6894 - val_loss: 0.0885 - val_accuracy: 0.9965 - val_iou_score: 0.8380
Epoch 260/300
84/84 [==============================] - 52s 614ms/step - loss: 0.1842 - accuracy: 0.9928 - iou_score: 0.6903 - val_loss: 0.0920 - val_accuracy: 0.9963 - val_iou_score: 0.8322
Epoch 261/300
84/84 [==============================] - 51s 610ms/step - loss: 0.1837 - accuracy: 0.9928 - iou_score: 0.6909 - val_loss: 0.0882 - val_accuracy: 0.9965 - val_iou_score: 0.8386
Epoch 262/300
84/84 [==============================] - 51s 608ms/step - loss: 0.1800 - accuracy: 0.9930 - iou_score: 0.6960 - val_loss: 0.0854 - val_accuracy: 0.9966 - val_iou_score: 0.8433
Epoch 263/300
84/84 [==============================] - 51s 612ms/step - loss: 0.1829 - accuracy: 0.9928 - iou_score: 0.6922 - val_loss: 0.1012 - val_accuracy: 0.9960 - val_iou_score: 0.8169
Epoch 264/300
84/84 [==============================] - 51s 605ms

# Evaluation

In [9]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")

# 2.4) Supervised Segmentation
A U-Net is trained here to perform a semantic segmentation of laser points in the LASTEN data set.

# Network
A U-Net based network is instantiated with keras to run a semantic segmentation.

In [10]:
model = UNet(filters=48, input_shape=(height, width, 1))
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 768, 768, 1) 0                                            
__________________________________________________________________________________________________
conv2d_18 (Conv2D)              (None, 768, 768, 48) 432         input_2[0][0]                    
__________________________________________________________________________________________________
batch_normalization_18 (BatchNo (None, 768, 768, 48) 192         conv2d_18[0][0]                  
__________________________________________________________________________________________________
activation_18 (Activation)      (None, 768, 768, 48) 0           batch_normalization_18[0][0]     
____________________________________________________________________________________________

# Data Preparation
The generator sequence for training is built.

In [11]:
image_ids = None

train_gen = LASTENSequence(path_train,
                           batch_size=2,
                           image_ids=image_ids,
                           preprocess_input=pre_une,
                           shuffle=True,
                           augment=True,
                           width=width,
                           height=height)

validation_gen = LASTENSequence(path_validation,
                                batch_size=2,
                                preprocess_input=pre_une,
                                width=width,
                                height=height)

#test_gen = LASTENSequence(path_train,
#                          batch_size=8,
#                          preprocess_input=pre_une,
#                          width=width,
#                          height=height)

# Training Preparation
Prepare the settings for training the model.

In [12]:
save_path = "experiments/2_4_segmentation"

logger = CSVLogger(save_path + "/result.log")
checker = ModelCheckpoint(save_path + "/weights.{epoch:02d}.hdf5",
                          period=10)

callbacks = [logger, checker]

model.compile(optimizer='adam',
              loss = dice_loss,
              metrics = ['accuracy', iou_score])

# Training
The model is fitted with a generator.

In [13]:
model.fit_generator(train_gen,
                    epochs=300,
                    callbacks=callbacks,
                    validation_data=validation_gen,
                    validation_freq=1,
                    use_multiprocessing=True,
                    workers=10,
                    max_queue_size=32)

Epoch 1/300
84/84 [==============================] - 79s 935ms/step - loss: 0.6626 - accuracy: 0.9572 - iou_score: 0.2107 - val_loss: 0.9623 - val_accuracy: 0.9809 - val_iou_score: 0.0192
Epoch 2/300
84/84 [==============================] - 65s 773ms/step - loss: 0.3436 - accuracy: 0.9896 - iou_score: 0.4902 - val_loss: 0.7955 - val_accuracy: 0.9832 - val_iou_score: 0.1164
Epoch 3/300
84/84 [==============================] - 65s 776ms/step - loss: 0.2705 - accuracy: 0.9903 - iou_score: 0.5746 - val_loss: 0.4643 - val_accuracy: 0.9866 - val_iou_score: 0.3885
Epoch 4/300
84/84 [==============================] - 65s 775ms/step - loss: 0.2494 - accuracy: 0.9907 - iou_score: 0.6012 - val_loss: 0.2598 - val_accuracy: 0.9913 - val_iou_score: 0.5881
Epoch 5/300
84/84 [==============================] - 65s 771ms/step - loss: 0.2400 - accuracy: 0.9909 - iou_score: 0.6134 - val_loss: 0.4099 - val_accuracy: 0.9889 - val_iou_score: 0.4211
Epoch 6/300
84/84 [==============================] - 65s 771

84/84 [==============================] - 66s 784ms/step - loss: 0.1951 - accuracy: 0.9923 - iou_score: 0.6745 - val_loss: 0.1019 - val_accuracy: 0.9960 - val_iou_score: 0.8156
Epoch 88/300
84/84 [==============================] - 66s 784ms/step - loss: 0.1954 - accuracy: 0.9923 - iou_score: 0.6744 - val_loss: 0.0993 - val_accuracy: 0.9962 - val_iou_score: 0.8200
Epoch 89/300
84/84 [==============================] - 66s 781ms/step - loss: 0.1967 - accuracy: 0.9923 - iou_score: 0.6723 - val_loss: 0.1251 - val_accuracy: 0.9950 - val_iou_score: 0.7787
Epoch 90/300
84/84 [==============================] - 67s 797ms/step - loss: 0.1926 - accuracy: 0.9924 - iou_score: 0.6780 - val_loss: 0.1136 - val_accuracy: 0.9955 - val_iou_score: 0.7966
Epoch 91/300
84/84 [==============================] - 66s 783ms/step - loss: 0.1954 - accuracy: 0.9923 - iou_score: 0.6744 - val_loss: 0.1049 - val_accuracy: 0.9959 - val_iou_score: 0.8109
Epoch 92/300
84/84 [==============================] - 66s 781ms/step

84/84 [==============================] - 66s 782ms/step - loss: 0.1878 - accuracy: 0.9926 - iou_score: 0.6855 - val_loss: 0.0927 - val_accuracy: 0.9964 - val_iou_score: 0.8311
Epoch 174/300
84/84 [==============================] - 66s 780ms/step - loss: 0.1944 - accuracy: 0.9924 - iou_score: 0.6756 - val_loss: 0.0860 - val_accuracy: 0.9966 - val_iou_score: 0.8424
Epoch 175/300
84/84 [==============================] - 66s 787ms/step - loss: 0.1895 - accuracy: 0.9926 - iou_score: 0.6828 - val_loss: 0.1003 - val_accuracy: 0.9960 - val_iou_score: 0.8183
Epoch 176/300
84/84 [==============================] - 66s 784ms/step - loss: 0.1876 - accuracy: 0.9926 - iou_score: 0.6855 - val_loss: 0.1058 - val_accuracy: 0.9958 - val_iou_score: 0.8093
Epoch 177/300
84/84 [==============================] - 66s 785ms/step - loss: 0.1864 - accuracy: 0.9927 - iou_score: 0.6869 - val_loss: 0.0970 - val_accuracy: 0.9962 - val_iou_score: 0.8238
Epoch 178/300
84/84 [==============================] - 66s 785ms

84/84 [==============================] - 66s 782ms/step - loss: 0.1846 - accuracy: 0.9928 - iou_score: 0.6895 - val_loss: 0.0915 - val_accuracy: 0.9964 - val_iou_score: 0.8331
Epoch 260/300
84/84 [==============================] - 66s 789ms/step - loss: 0.1861 - accuracy: 0.9927 - iou_score: 0.6873 - val_loss: 0.0841 - val_accuracy: 0.9967 - val_iou_score: 0.8455
Epoch 261/300
84/84 [==============================] - 66s 783ms/step - loss: 0.1805 - accuracy: 0.9929 - iou_score: 0.6957 - val_loss: 0.0993 - val_accuracy: 0.9961 - val_iou_score: 0.8201
Epoch 262/300
84/84 [==============================] - 66s 784ms/step - loss: 0.1849 - accuracy: 0.9928 - iou_score: 0.6892 - val_loss: 0.0832 - val_accuracy: 0.9967 - val_iou_score: 0.8471
Epoch 263/300
84/84 [==============================] - 66s 787ms/step - loss: 0.1828 - accuracy: 0.9928 - iou_score: 0.6922 - val_loss: 0.0862 - val_accuracy: 0.9966 - val_iou_score: 0.8420
Epoch 264/300
84/84 [==============================] - 66s 787ms

# Evaluation

In [14]:
X, y = validation_gen[0]
y_pred = model.predict(X)

plt.imsave(save_path + "/y.png",y[0,:,:,0], cmap="gray")
plt.imsave(save_path + "/y_pred.png",y_pred[0,:,:,0], cmap="gray")